# Part 1 - A description of the problem and a discussion of the background.

# Description of the Problem

In Canoas a city of the Southern Part of Brazil, if someone is looking to categorize all the cheese burguers house and slice into neighborhood


shop near the principals avenues, what are the best spot to open it?


# Discussion of the Background

My client are looking for a good spot to install his coffee shop. It must be profitable, a well looked place with many potencial customers, far from another coffee shop and near from a bar.

We have to assume the customers must passing through the coffee shop to go to work, and they are looking for a good spot to take a break when they got tired from work.

The location of the coffee has a significant impact on the returns.

# Part 2 - A description of the data and how it will be used to solve the problem

# Description of the Data

This project will rely on public data from Foursquare located in [Cafeterias em Porto Alegre](https://pt.foursquare.com/explore?cat=coffee&mode=url&near=Porto%20Alegre%2C%20RS%2C%20Brasil&nearGeoId=72057594041380861) and [Bares em Porto Alegre](https://pt.foursquare.com/explore?mode=url&near=Porto%20Alegre%2C%20RS%2C%20Brasil&nearGeoId=72057594041380861&q=bar)

To explore and target recommended locations across different venues according to the presence of similar facilities, we will access data through FourSquare API 
interface and arrange them as a dataframe for visualization.

# Methodology section

The Methodology section will describe the main components of our analysis and predication system. The Methodology section comprises four stages:

1. Collect Inspection Data
2. Explore and Understand Data
3. Data preparation and preprocessing 
4. Modeling

### 1. Collect Inspection Data

In this stage I'm installing and importing the libraries

In [ ]:
!conda install -c conda-forge geopy --yes 
!conda install -c conda-forge folium=0.5.0 --yes

In [91]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize
import folium # plotting library

from folium.plugins import HeatMap


print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.



Setting my account details to develop a valid code

In [2]:
CLIENT_ID = 'DBD1G4TA4ZJX0OLGFLA2VRVSQ11YJAW34TAZTEHGM4D0PKX1' # your Foursquare ID
CLIENT_SECRET = 'MOBFAZIUVBVLOQUOTY1SKZ3PVEPJKKM3ZWR4U3DRVADRIMMB' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 20
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: DBD1G4TA4ZJX0OLGFLA2VRVSQ11YJAW34TAZTEHGM4D0PKX1
CLIENT_SECRET:MOBFAZIUVBVLOQUOTY1SKZ3PVEPJKKM3ZWR4U3DRVADRIMMB


#### Let's assume we are in Porto Alegre, let's get the latitude and longitude coordinates.

In [3]:
latitude = -30.027769
longitude = -51.227775


print(latitude, longitude)

-30.027769 -51.227775


### 2. Explore and understand data

##### Now, let's assume that what we want is a coffee shop.
So, let's define a query to search for coffee shop that is within 1000 meters near from Porto Alegre.

In [4]:
search_query = 'Cafeteria'
radius = 1000
print(search_query + ' .... OK!')

Cafeteria .... OK!


In [5]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=DBD1G4TA4ZJX0OLGFLA2VRVSQ11YJAW34TAZTEHGM4D0PKX1&client_secret=MOBFAZIUVBVLOQUOTY1SKZ3PVEPJKKM3ZWR4U3DRVADRIMMB&ll=-30.027769,-51.227775&v=20180604&query=Cafeteria&radius=1000&limit=20'

In [6]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ea86779bae9a2001b21b02b'},
 'response': {'venues': [{'id': '4d4c542d7929594187064fc7',
    'name': 'Cafeteria Algo a Mais',
    'location': {'address': 'Subsolo Malcon',
     'lat': -30.02974262120043,
     'lng': -51.226115430534534,
     'labeledLatLngs': [{'label': 'display',
       'lat': -30.02974262120043,
       'lng': -51.226115430534534}],
     'distance': 271,
     'cc': 'BR',
     'city': 'Porto Alegre',
     'state': 'RS',
     'country': 'Brasil',
     'formattedAddress': ['Subsolo Malcon', 'Porto Alegre, RS', 'Brasil']},
    'categories': [{'id': '4bf58dd8d48988d16d941735',
      'name': 'Café',
      'pluralName': 'Cafés',
      'shortName': 'Café',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/cafe_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1588095214',
    'hasPerk': False},
   {'id': '511558d8e4b0ce6dc580cddd',
    'name': 'Cafeteria Fênix',
    'location': {'address': 'Galer

##### Let's create a dataframe from the data we collect

In [7]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """


,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.cc,location.city,location.state,location.country,location.formattedAddress,location.crossStreet,location.postalCode,location.neighborhood,venuePage.id
0,4d4c542d7929594187064fc7,Cafeteria Algo a Mais,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",v-1588095214,False,Subsolo Malcon,-30.029743,-51.226115,"[{'label': 'display', 'lat': -30.0297426212004...",271,BR,Porto Alegre,RS,Brasil,"[Subsolo Malcon, Porto Alegre, RS, Brasil]",NaN,NaN,NaN,NaN
1,511558d8e4b0ce6dc580cddd,Cafeteria Fênix,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",v-1588095214,False,Galeria Lamberts,-30.026341,-51.225702,"[{'label': 'display', 'lat': -30.0263412365417...",255,BR,Porto Alegre,RS,Brasil,"[Galeria Lamberts (Loja 31/33), Porto Alegre, ...",Loja 31/33,90030-000,NaN,NaN
2,4eb3e5d8be7bfc28499cdd75,Cafeteria do Hospital Santa Rita,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",v-1588095214,False,Hospital Santa Rita,-30.030656,-51.220473,"[{'label': 'display', 'lat': -30.0306562320857...",773,BR,Porto Alegre,RS,Brasil,"[Hospital Santa Rita, Porto Alegre, RS, 90035-...",NaN,90035-074,NaN,NaN
3,51cf2865498ee1b893cebded,Cafeteria Mezanino,[],v-1588095214,False,Centro Cultural CEEE Érico Veríssimo,-30.029709,-51.227869,"[{'label': 'display', 'lat': -30.0297089711154...",216,BR,Porto Alegre,RS,Brasil,"[Centro Cultural CEEE Érico Veríssimo, Porto A...",NaN,NaN,NaN,NaN
4,54de4e7b498e49208265aa4c,Cafeteria Santa Helena,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",v-1588095214,False,"R. dos Andradas, 1560, Lj. 16",-30.029786,-51.225947,"[{'label': 'display', 'lat': -30.0297855693904...",285,BR,Porto Alegre,RS,Brasil,"[R. dos Andradas, 1560, Lj. 16 (Galeria Malcon...",Galeria Malcon,NaN,NaN,NaN


In [8]:
dataframe.shape

(20, 19)

In [9]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,lat,lng,labeledLatLngs,distance,cc,city,state,country,formattedAddress,crossStreet,postalCode,neighborhood,id
0,Cafeteria Algo a Mais,Café,Subsolo Malcon,-30.029743,-51.226115,"[{'label': 'display', 'lat': -30.0297426212004...",271,BR,Porto Alegre,RS,Brasil,"[Subsolo Malcon, Porto Alegre, RS, Brasil]",NaN,NaN,NaN,4d4c542d7929594187064fc7
1,Cafeteria Fênix,Café,Galeria Lamberts,-30.026341,-51.225702,"[{'label': 'display', 'lat': -30.0263412365417...",255,BR,Porto Alegre,RS,Brasil,"[Galeria Lamberts (Loja 31/33), Porto Alegre, ...",Loja 31/33,90030-000,NaN,511558d8e4b0ce6dc580cddd
2,Cafeteria do Hospital Santa Rita,Café,Hospital Santa Rita,-30.030656,-51.220473,"[{'label': 'display', 'lat': -30.0306562320857...",773,BR,Porto Alegre,RS,Brasil,"[Hospital Santa Rita, Porto Alegre, RS, 90035-...",NaN,90035-074,NaN,4eb3e5d8be7bfc28499cdd75
3,Cafeteria Mezanino,None,Centro Cultural CEEE Érico Veríssimo,-30.029709,-51.227869,"[{'label': 'display', 'lat': -30.0297089711154...",216,BR,Porto Alegre,RS,Brasil,"[Centro Cultural CEEE Érico Veríssimo, Porto A...",NaN,NaN,NaN,51cf2865498ee1b893cebded
4,Cafeteria Santa Helena,Coffee Shop,"R. dos Andradas, 1560, Lj. 16",-30.029786,-51.225947,"[{'label': 'display', 'lat': -30.0297855693904...",285,BR,Porto Alegre,RS,Brasil,"[R. dos Andradas, 1560, Lj. 16 (Galeria Malcon...",Galeria Malcon,NaN,NaN,54de4e7b498e49208265aa4c
5,Cafeteria - 5 andar,Café,Rua Caldas Júnior 108,-30.029111,-51.232381,"[{'label': 'display', 'lat': -30.0291110996504...",468,BR,Porto Alegre,RS,Brasil,"[Rua Caldas Júnior 108 (Siqueira Campos), Port...",Siqueira Campos,NaN,NaN,4e5e186ad1640a5975c4b9f2
6,Cafeteria Universitaria,Snack Place,Rua Vigário José Inácio,-30.027983,-51.223101,"[{'label': 'display', 'lat': -30.0279825751354...",451,BR,Porto Alegre,RS,Brasil,[Rua Vigário José Inácio (Avenida Voluntários ...,Avenida Voluntários Da Pátria,NaN,NaN,564f7959498e86dce30d2d1c
7,Cafeteria e Tabacaria TC 35,Café,"R. Senhor dos Passos, 241",-30.029189,-51.222866,"[{'label': 'display', 'lat': -30.0291893149861...",498,BR,Porto Alegre,RS,Brasil,"[R. Senhor dos Passos, 241, Porto Alegre, RS, ...",NaN,NaN,NaN,4f565ee7e4b0861b32b03740
8,Cafeteria Factum,Coffee Shop,"Lg. João Amorim de Albuquerque, 60",-30.032100,-51.229465,"[{'label': 'display', 'lat': -30.0321, 'lng': ...",508,BR,Porto Alegre,RS,Brasil,"[Lg. João Amorim de Albuquerque, 60 (Faculdade...",Faculdade Factum,90010-250,Centro Histórico,5591409b498ec31df9b70cd4
9,Cafeteria Assembleia,Café,NaN,-30.033112,-51.231334,"[{'label': 'display', 'lat': -30.0331122260256...",686,BR,Porto Alegre,RS,Brasil,"[Porto Alegre, RS, Brasil]",NaN,NaN,NaN,511e6006e4b058afc3a92b95


In [10]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=15)
# generate map centered around Porto Alegre

# add the Cafeterias restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

    
    
# display map
venues_map

### These points are the coffee shops nearby Porto Alegre Downtown.

Now, let's find the top 20 bars in Porto Alegre and observe if are coffee shops nearby.

In [11]:
search_query = 'Bar'
radius = 1000
print(search_query + ' .... OK!')

Bar .... OK!


In [12]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=DBD1G4TA4ZJX0OLGFLA2VRVSQ11YJAW34TAZTEHGM4D0PKX1&client_secret=MOBFAZIUVBVLOQUOTY1SKZ3PVEPJKKM3ZWR4U3DRVADRIMMB&ll=-30.027769,-51.227775&v=20180604&query=Bar&radius=1000&limit=20'

In [13]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ea8672add0f85002256104e'},
 'response': {'venues': [{'id': '5346d09f498edd8d3cfb2ed0',
    'name': 'Bar Chopp 26',
    'location': {'address': 'Mercado Público',
     'crossStreet': 'Quadrante 1, Loja 46',
     'lat': -30.027556391239994,
     'lng': -51.22791883201101,
     'labeledLatLngs': [{'label': 'display',
       'lat': -30.027556391239994,
       'lng': -51.22791883201101}],
     'distance': 27,
     'postalCode': '90020-070',
     'cc': 'BR',
     'neighborhood': 'Centro Histórico',
     'city': 'Porto Alegre',
     'state': 'RS',
     'country': 'Brasil',
     'formattedAddress': ['Mercado Público (Quadrante 1, Loja 46)',
      'Porto Alegre, RS',
      '90020-070',
      'Brasil']},
    'categories': [{'id': '4bf58dd8d48988d16b941735',
      'name': 'Brazilian Restaurant',
      'pluralName': 'Brazilian Restaurants',
      'shortName': 'Brazilian',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/argentinian_',
    

In [14]:
# assign relevant part of JSON to venues
venues2= results['response']['venues']

# tranform venues into a dataframe
dataframe2 = json_normalize(venues2)
dataframe2.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """


,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.neighborhood,location.city,location.state,location.country,location.formattedAddress
0,5346d09f498edd8d3cfb2ed0,Bar Chopp 26,"[{'id': '4bf58dd8d48988d16b941735', 'name': 'B...",v-1588095225,False,Mercado Público,"Quadrante 1, Loja 46",-30.027556,-51.227919,"[{'label': 'display', 'lat': -30.0275563912399...",27,90020-070,BR,Centro Histórico,Porto Alegre,RS,Brasil,"[Mercado Público (Quadrante 1, Loja 46), Porto..."
1,567593fb498eeaae9613f292,Bar e Restaurante Velloso,"[{'id': '52939ad03cf9994f4e043a3a', 'name': 'S...",v-1588095225,False,Av. Júlio de Castilhos,NaN,-30.026105,-51.225257,"[{'label': 'display', 'lat': -30.0261052726659...",305,90030-131,BR,Centro Histórico,Porto Alegre,RS,Brasil,"[Av. Júlio de Castilhos, Porto Alegre, RS, 900..."
2,4bfad2f365fbc9b63094906c,Tuim Bar & Choperia,"[{'id': '4bf58dd8d48988d116941735', 'name': 'B...",v-1588095225,False,"R. Gen. Câmara, 333",NaN,-30.030401,-51.229848,"[{'label': 'display', 'lat': -30.0304010913479...",354,90010-230,BR,NaN,Porto Alegre,RS,Brasil,"[R. Gen. Câmara, 333, Porto Alegre, RS, 90010-..."
3,4eb40cf69a52ad1e3e53f81e,Tutty's Bar e Restaurante,"[{'id': '4bf58dd8d48988d16b941735', 'name': 'B...",v-1588095225,False,"Av. Independência, 30",Lj. 15,-30.029363,-51.221338,"[{'label': 'display', 'lat': -30.0293632228157...",645,90035-070,BR,NaN,Porto Alegre,RS,Brasil,"[Av. Independência, 30 (Lj. 15), Porto Alegre,..."
4,4cc9ffcd1ce5236a7374bc65,Bar e Restaurante Refúgio's,"[{'id': '4bf58dd8d48988d16b941735', 'name': 'B...",v-1588095225,False,"R. dos Andradas, 717",NaN,-30.031505,-51.234517,"[{'label': 'display', 'lat': -30.0315049316440...",771,90020-004,BR,NaN,Porto Alegre,RS,Brasil,"[R. dos Andradas, 717, Porto Alegre, RS, 90020..."


In [15]:
dataframe2.shape

(20, 18)

In [16]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns2 = ['name', 'categories'] + [col for col in dataframe2.columns if col.startswith('location.')] + ['id']
dataframe_filtered2 = dataframe2.loc[:, filtered_columns2]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered2['categories'] = dataframe_filtered2.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered2.columns = [column.split('.')[-1] for column in dataframe_filtered2.columns]

dataframe_filtered2

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,neighborhood,city,state,country,formattedAddress,id
0,Bar Chopp 26,Brazilian Restaurant,Mercado Público,"Quadrante 1, Loja 46",-30.027556,-51.227919,"[{'label': 'display', 'lat': -30.0275563912399...",27,90020-070,BR,Centro Histórico,Porto Alegre,RS,Brasil,"[Mercado Público (Quadrante 1, Loja 46), Porto...",5346d09f498edd8d3cfb2ed0
1,Bar e Restaurante Velloso,Southern Brazilian Restaurant,Av. Júlio de Castilhos,NaN,-30.026105,-51.225257,"[{'label': 'display', 'lat': -30.0261052726659...",305,90030-131,BR,Centro Histórico,Porto Alegre,RS,Brasil,"[Av. Júlio de Castilhos, Porto Alegre, RS, 900...",567593fb498eeaae9613f292
2,Tuim Bar & Choperia,Bar,"R. Gen. Câmara, 333",NaN,-30.030401,-51.229848,"[{'label': 'display', 'lat': -30.0304010913479...",354,90010-230,BR,NaN,Porto Alegre,RS,Brasil,"[R. Gen. Câmara, 333, Porto Alegre, RS, 90010-...",4bfad2f365fbc9b63094906c
3,Tutty's Bar e Restaurante,Brazilian Restaurant,"Av. Independência, 30",Lj. 15,-30.029363,-51.221338,"[{'label': 'display', 'lat': -30.0293632228157...",645,90035-070,BR,NaN,Porto Alegre,RS,Brasil,"[Av. Independência, 30 (Lj. 15), Porto Alegre,...",4eb40cf69a52ad1e3e53f81e
4,Bar e Restaurante Refúgio's,Brazilian Restaurant,"R. dos Andradas, 717",NaN,-30.031505,-51.234517,"[{'label': 'display', 'lat': -30.0315049316440...",771,90020-004,BR,NaN,Porto Alegre,RS,Brasil,"[R. dos Andradas, 717, Porto Alegre, RS, 90020...",4cc9ffcd1ce5236a7374bc65
5,Bar e Lancheria Alto Astral,Fast Food Restaurant,"R. dos Andradas, 666",R. Gen. Bento Martins,-30.031983,-51.234843,"[{'label': 'display', 'lat': -30.0319828179324...",827,NaN,BR,Centro Histórico,Porto Alegre,RS,Brasil,"[R. dos Andradas, 666 (R. Gen. Bento Martins),...",4cc328073d7fa1cd12c7a35f
6,Madrugas Bar,Snack Place,"Pç. Conde de Porto Alegre, 359",NaN,-30.031822,-51.224036,"[{'label': 'display', 'lat': -30.0318217010288...",577,95880-000,BR,NaN,Porto Alegre,RS,Brasil,"[Pç. Conde de Porto Alegre, 359, Porto Alegre,...",509d4941e4b00baa8beee001
7,Dinápoli Restaurante Bar,Brazilian Restaurant,"R. dos Andradas, 816",NaN,-30.031111,-51.233417,"[{'label': 'display', 'lat': -30.0311108998415...",658,90020-004,BR,Centro Histórico,Porto Alegre,RS,Brasil,"[R. dos Andradas, 816, Porto Alegre, RS, 90020...",50c76ea1e4b0480110f79012
8,514 Lobby Bar e restaurante,Italian Restaurant,NaN,NaN,-30.028009,-51.221608,"[{'label': 'display', 'lat': -30.0280092221616...",594,NaN,BR,NaN,Porto Alegre,RS,Brasil,"[Porto Alegre, RS, Brasil]",4ce30cf6438b224b2b3c80a3
9,Bar Chopp Havana Restaurante,Bar,Mercado Público,"Quadrante 4, Lojas 127/129",-30.027806,-51.227461,"[{'label': 'display', 'lat': -30.0278059279554...",30,90020-070,BR,Centro Histórico,Porto Alegre,RS,Brasil,"[Mercado Público (Quadrante 4, Lojas 127/129),...",4e15db488877cd57121198ca


In [17]:
venues_map2 = folium.Map(location=[latitude, longitude], zoom_start=15)
# generate map centered around Porto Alegre

# add the bars restaurants as green circle markers
for lat, lng, label in zip(dataframe_filtered2.lat, dataframe_filtered2.lng, dataframe_filtered2.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='green',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map2)
    
    
# display map
venues_map2

### Now lets plot the map containing the bars and cafeterias in Porto Alegre

In [18]:
venues_map3 = folium.Map(location=[latitude, longitude], zoom_start=15)
# generate map centered around Porto Alegre

# add the bars restaurants as green circle markers
for lat, lng, label in zip(dataframe_filtered2.lat, dataframe_filtered2.lng, dataframe_filtered2.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='green',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map3)
    

# add the Cafeterias restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map3)
    
    
    
    
# display map
venues_map3

### 3. Data preparation and preprocessing data

### On this part we are going to work with the generated data

Lets call the dataframe created with data scrapped from foursquare 'Bares e cafeterias'.

In [19]:
df_bares = dataframe_filtered2
df_bares

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,neighborhood,city,state,country,formattedAddress,id
0,Bar Chopp 26,Brazilian Restaurant,Mercado Público,"Quadrante 1, Loja 46",-30.027556,-51.227919,"[{'label': 'display', 'lat': -30.0275563912399...",27,90020-070,BR,Centro Histórico,Porto Alegre,RS,Brasil,"[Mercado Público (Quadrante 1, Loja 46), Porto...",5346d09f498edd8d3cfb2ed0
1,Bar e Restaurante Velloso,Southern Brazilian Restaurant,Av. Júlio de Castilhos,NaN,-30.026105,-51.225257,"[{'label': 'display', 'lat': -30.0261052726659...",305,90030-131,BR,Centro Histórico,Porto Alegre,RS,Brasil,"[Av. Júlio de Castilhos, Porto Alegre, RS, 900...",567593fb498eeaae9613f292
2,Tuim Bar & Choperia,Bar,"R. Gen. Câmara, 333",NaN,-30.030401,-51.229848,"[{'label': 'display', 'lat': -30.0304010913479...",354,90010-230,BR,NaN,Porto Alegre,RS,Brasil,"[R. Gen. Câmara, 333, Porto Alegre, RS, 90010-...",4bfad2f365fbc9b63094906c
3,Tutty's Bar e Restaurante,Brazilian Restaurant,"Av. Independência, 30",Lj. 15,-30.029363,-51.221338,"[{'label': 'display', 'lat': -30.0293632228157...",645,90035-070,BR,NaN,Porto Alegre,RS,Brasil,"[Av. Independência, 30 (Lj. 15), Porto Alegre,...",4eb40cf69a52ad1e3e53f81e
4,Bar e Restaurante Refúgio's,Brazilian Restaurant,"R. dos Andradas, 717",NaN,-30.031505,-51.234517,"[{'label': 'display', 'lat': -30.0315049316440...",771,90020-004,BR,NaN,Porto Alegre,RS,Brasil,"[R. dos Andradas, 717, Porto Alegre, RS, 90020...",4cc9ffcd1ce5236a7374bc65
5,Bar e Lancheria Alto Astral,Fast Food Restaurant,"R. dos Andradas, 666",R. Gen. Bento Martins,-30.031983,-51.234843,"[{'label': 'display', 'lat': -30.0319828179324...",827,NaN,BR,Centro Histórico,Porto Alegre,RS,Brasil,"[R. dos Andradas, 666 (R. Gen. Bento Martins),...",4cc328073d7fa1cd12c7a35f
6,Madrugas Bar,Snack Place,"Pç. Conde de Porto Alegre, 359",NaN,-30.031822,-51.224036,"[{'label': 'display', 'lat': -30.0318217010288...",577,95880-000,BR,NaN,Porto Alegre,RS,Brasil,"[Pç. Conde de Porto Alegre, 359, Porto Alegre,...",509d4941e4b00baa8beee001
7,Dinápoli Restaurante Bar,Brazilian Restaurant,"R. dos Andradas, 816",NaN,-30.031111,-51.233417,"[{'label': 'display', 'lat': -30.0311108998415...",658,90020-004,BR,Centro Histórico,Porto Alegre,RS,Brasil,"[R. dos Andradas, 816, Porto Alegre, RS, 90020...",50c76ea1e4b0480110f79012
8,514 Lobby Bar e restaurante,Italian Restaurant,NaN,NaN,-30.028009,-51.221608,"[{'label': 'display', 'lat': -30.0280092221616...",594,NaN,BR,NaN,Porto Alegre,RS,Brasil,"[Porto Alegre, RS, Brasil]",4ce30cf6438b224b2b3c80a3
9,Bar Chopp Havana Restaurante,Bar,Mercado Público,"Quadrante 4, Lojas 127/129",-30.027806,-51.227461,"[{'label': 'display', 'lat': -30.0278059279554...",30,90020-070,BR,Centro Histórico,Porto Alegre,RS,Brasil,"[Mercado Público (Quadrante 4, Lojas 127/129),...",4e15db488877cd57121198ca


In [20]:
df_bares.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   name              20 non-null     object 
 1   categories        20 non-null     object 
 2   address           19 non-null     object 
 3   crossStreet       7 non-null      object 
 4   lat               20 non-null     float64
 5   lng               20 non-null     float64
 6   labeledLatLngs    20 non-null     object 
 7   distance          20 non-null     int64  
 8   postalCode        15 non-null     object 
 9   cc                20 non-null     object 
 10  neighborhood      7 non-null      object 
 11  city              20 non-null     object 
 12  state             20 non-null     object 
 13  country           20 non-null     object 
 14  formattedAddress  20 non-null     object 
 15  id                20 non-null     object 
dtypes: float64(2), int64(1), object(13)
memory usa

In [21]:
bares_simp = df_bares.drop(columns=['name', 'categories','address','crossStreet','labeledLatLngs','distance','postalCode','cc', 'neighborhood', 'city', 'state', 'country', 'formattedAddress', 'id'], axis=1, inplace=False)

In [22]:
bares_simp.head()

,lat,lng
0,-30.027556,-51.227919
1,-30.026105,-51.225257
2,-30.030401,-51.229848
3,-30.029363,-51.221338
4,-30.031505,-51.234517


After all i've created a dataframe using only the geographical coordinates from 'bares', and do the same from 'cafeterias'.

In [23]:
df_cafes = dataframe_filtered
df_cafes

,name,categories,address,lat,lng,labeledLatLngs,distance,cc,city,state,country,formattedAddress,crossStreet,postalCode,neighborhood,id
0,Cafeteria Algo a Mais,Café,Subsolo Malcon,-30.029743,-51.226115,"[{'label': 'display', 'lat': -30.0297426212004...",271,BR,Porto Alegre,RS,Brasil,"[Subsolo Malcon, Porto Alegre, RS, Brasil]",NaN,NaN,NaN,4d4c542d7929594187064fc7
1,Cafeteria Fênix,Café,Galeria Lamberts,-30.026341,-51.225702,"[{'label': 'display', 'lat': -30.0263412365417...",255,BR,Porto Alegre,RS,Brasil,"[Galeria Lamberts (Loja 31/33), Porto Alegre, ...",Loja 31/33,90030-000,NaN,511558d8e4b0ce6dc580cddd
2,Cafeteria do Hospital Santa Rita,Café,Hospital Santa Rita,-30.030656,-51.220473,"[{'label': 'display', 'lat': -30.0306562320857...",773,BR,Porto Alegre,RS,Brasil,"[Hospital Santa Rita, Porto Alegre, RS, 90035-...",NaN,90035-074,NaN,4eb3e5d8be7bfc28499cdd75
3,Cafeteria Mezanino,None,Centro Cultural CEEE Érico Veríssimo,-30.029709,-51.227869,"[{'label': 'display', 'lat': -30.0297089711154...",216,BR,Porto Alegre,RS,Brasil,"[Centro Cultural CEEE Érico Veríssimo, Porto A...",NaN,NaN,NaN,51cf2865498ee1b893cebded
4,Cafeteria Santa Helena,Coffee Shop,"R. dos Andradas, 1560, Lj. 16",-30.029786,-51.225947,"[{'label': 'display', 'lat': -30.0297855693904...",285,BR,Porto Alegre,RS,Brasil,"[R. dos Andradas, 1560, Lj. 16 (Galeria Malcon...",Galeria Malcon,NaN,NaN,54de4e7b498e49208265aa4c
5,Cafeteria - 5 andar,Café,Rua Caldas Júnior 108,-30.029111,-51.232381,"[{'label': 'display', 'lat': -30.0291110996504...",468,BR,Porto Alegre,RS,Brasil,"[Rua Caldas Júnior 108 (Siqueira Campos), Port...",Siqueira Campos,NaN,NaN,4e5e186ad1640a5975c4b9f2
6,Cafeteria Universitaria,Snack Place,Rua Vigário José Inácio,-30.027983,-51.223101,"[{'label': 'display', 'lat': -30.0279825751354...",451,BR,Porto Alegre,RS,Brasil,[Rua Vigário José Inácio (Avenida Voluntários ...,Avenida Voluntários Da Pátria,NaN,NaN,564f7959498e86dce30d2d1c
7,Cafeteria e Tabacaria TC 35,Café,"R. Senhor dos Passos, 241",-30.029189,-51.222866,"[{'label': 'display', 'lat': -30.0291893149861...",498,BR,Porto Alegre,RS,Brasil,"[R. Senhor dos Passos, 241, Porto Alegre, RS, ...",NaN,NaN,NaN,4f565ee7e4b0861b32b03740
8,Cafeteria Factum,Coffee Shop,"Lg. João Amorim de Albuquerque, 60",-30.032100,-51.229465,"[{'label': 'display', 'lat': -30.0321, 'lng': ...",508,BR,Porto Alegre,RS,Brasil,"[Lg. João Amorim de Albuquerque, 60 (Faculdade...",Faculdade Factum,90010-250,Centro Histórico,5591409b498ec31df9b70cd4
9,Cafeteria Assembleia,Café,NaN,-30.033112,-51.231334,"[{'label': 'display', 'lat': -30.0331122260256...",686,BR,Porto Alegre,RS,Brasil,"[Porto Alegre, RS, Brasil]",NaN,NaN,NaN,511e6006e4b058afc3a92b95


In [24]:
cafes_simp = df_cafes.drop(columns=['name', 'categories','address','crossStreet','labeledLatLngs','distance','postalCode','cc', 'neighborhood', 'city', 'state', 'country', 'formattedAddress', 'id'], axis=1, inplace=False)


In [25]:
df_cafes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   name              20 non-null     object 
 1   categories        19 non-null     object 
 2   address           12 non-null     object 
 3   lat               20 non-null     float64
 4   lng               20 non-null     float64
 5   labeledLatLngs    20 non-null     object 
 6   distance          20 non-null     int64  
 7   cc                20 non-null     object 
 8   city              17 non-null     object 
 9   state             17 non-null     object 
 10  country           20 non-null     object 
 11  formattedAddress  20 non-null     object 
 12  crossStreet       6 non-null      object 
 13  postalCode        3 non-null      object 
 14  neighborhood      1 non-null      object 
 15  id                20 non-null     object 
dtypes: float64(2), int64(1), object(13)
memory usa

In [26]:
df_cafes.drop(columns=['name', 'categories','address','crossStreet','labeledLatLngs','distance','postalCode','cc', 'neighborhood', 'city', 'state', 'country', 'formattedAddress', 'id'], axis=1, inplace=True)

In [27]:
df_cafes

,lat,lng
0,-30.029743,-51.226115
1,-30.026341,-51.225702
2,-30.030656,-51.220473
3,-30.029709,-51.227869
4,-30.029786,-51.225947
5,-30.029111,-51.232381
6,-30.027983,-51.223101
7,-30.029189,-51.222866
8,-30.032100,-51.229465
9,-30.033112,-51.231334


In [28]:
array_cafes = df_cafes.to_numpy()
array_cafes

array([[-30.02974262, -51.22611543],
       [-30.02634124, -51.22570196],
       [-30.03065623, -51.22047272],
       [-30.02970897, -51.2278689 ],
       [-30.02978557, -51.22594727],
       [-30.0291111 , -51.23238115],
       [-30.02798258, -51.2231007 ],
       [-30.02918931, -51.22286562],
       [-30.0321    , -51.229465  ],
       [-30.03311223, -51.2313341 ],
       [-30.0281    , -51.222093  ],
       [-30.03127764, -51.22257433],
       [-30.03125132, -51.22181289],
       [-30.0300689 , -51.22018783],
       [-30.02722097, -51.22431699],
       [-30.032092  , -51.227013  ],
       [-30.03008877, -51.21775032],
       [-30.02957281, -51.22294739],
       [-30.036065  , -51.221949  ],
       [-30.02947681, -51.21778233]])

In [29]:
bares_simp

,lat,lng
0,-30.027556,-51.227919
1,-30.026105,-51.225257
2,-30.030401,-51.229848
3,-30.029363,-51.221338
4,-30.031505,-51.234517
5,-30.031983,-51.234843
6,-30.031822,-51.224036
7,-30.031111,-51.233417
8,-30.028009,-51.221608
9,-30.027806,-51.227461


In [30]:
array_bares = bares_simp.to_numpy()
array_bares

array([[-30.02755639, -51.22791883],
       [-30.02610527, -51.22525733],
       [-30.03040109, -51.22984754],
       [-30.02936322, -51.22133768],
       [-30.03150493, -51.2345171 ],
       [-30.03198282, -51.23484346],
       [-30.0318217 , -51.22403591],
       [-30.0311109 , -51.23341661],
       [-30.02800922, -51.22160848],
       [-30.02780593, -51.22746126],
       [-30.0331829 , -51.22218594],
       [-30.03267011, -51.22061198],
       [-30.03670548, -51.23193351],
       [-30.03608821, -51.22929901],
       [-30.03567   , -51.227152  ],
       [-30.036276  , -51.224133  ],
       [-30.03995959, -51.23073611],
       [-30.027119  , -51.2278    ],
       [-30.027118  , -51.227969  ],
       [-30.02715   , -51.227135  ]])

In [31]:
b = array_bares
b

array([[-30.02755639, -51.22791883],
       [-30.02610527, -51.22525733],
       [-30.03040109, -51.22984754],
       [-30.02936322, -51.22133768],
       [-30.03150493, -51.2345171 ],
       [-30.03198282, -51.23484346],
       [-30.0318217 , -51.22403591],
       [-30.0311109 , -51.23341661],
       [-30.02800922, -51.22160848],
       [-30.02780593, -51.22746126],
       [-30.0331829 , -51.22218594],
       [-30.03267011, -51.22061198],
       [-30.03670548, -51.23193351],
       [-30.03608821, -51.22929901],
       [-30.03567   , -51.227152  ],
       [-30.036276  , -51.224133  ],
       [-30.03995959, -51.23073611],
       [-30.027119  , -51.2278    ],
       [-30.027118  , -51.227969  ],
       [-30.02715   , -51.227135  ]])

In [32]:
c = array_cafes
c

array([[-30.02974262, -51.22611543],
       [-30.02634124, -51.22570196],
       [-30.03065623, -51.22047272],
       [-30.02970897, -51.2278689 ],
       [-30.02978557, -51.22594727],
       [-30.0291111 , -51.23238115],
       [-30.02798258, -51.2231007 ],
       [-30.02918931, -51.22286562],
       [-30.0321    , -51.229465  ],
       [-30.03311223, -51.2313341 ],
       [-30.0281    , -51.222093  ],
       [-30.03127764, -51.22257433],
       [-30.03125132, -51.22181289],
       [-30.0300689 , -51.22018783],
       [-30.02722097, -51.22431699],
       [-30.032092  , -51.227013  ],
       [-30.03008877, -51.21775032],
       [-30.02957281, -51.22294739],
       [-30.036065  , -51.221949  ],
       [-30.02947681, -51.21778233]])

### After that we generated 2 arrays to store the coordinates from bares and cafeterias, we are going to use this data do fit and score the kernel

#### Now lets create our data to test the fit, i'll select a few points in map using the below code, i'm trying to generate some points

In [33]:
xmin = -30.035505
xmax = -30.022946
ymin = -51.22057
ymax = -51.220806
grid_size = 0.001

In [34]:
def construct_grids(xmin, xmax, ymin, ymax, grid_size):

    # x,y coordinates for corner cells
    xmin = -51.2370 + grid_size
    xmax = xmin + (17 * grid_size)
    ymin = -30.0365 + grid_size
    ymax = ymin + (17 * grid_size)

    # x coordinates of the grid cells
    xgrid = np.arange(xmin, xmax, grid_size)
    # y coordinates of the grid cells
    ygrid = np.arange(ymin, ymax, grid_size)

    return (xgrid, ygrid)



#                    y          x
# diag inf esq -30.0405, -51.2370
# diag sup esq -30.0235, -51.2370
# diag inf dir -30.0405, -51.2200
# diag sup dir -30.0235, -51.2200
# 0.001


In [35]:
construct_grids(-51.2370, -51.2200, -30.0405, -30.0235, 0.001)

(array([-51.236, -51.235, -51.234, -51.233, -51.232, -51.231, -51.23 ,
        -51.229, -51.228, -51.227, -51.226, -51.225, -51.224, -51.223,
        -51.222, -51.221, -51.22 , -51.219]),
 array([-30.0355, -30.0345, -30.0335, -30.0325, -30.0315, -30.0305,
        -30.0295, -30.0285, -30.0275, -30.0265, -30.0255, -30.0245,
        -30.0235, -30.0225, -30.0215, -30.0205, -30.0195]))

In [36]:
x_pontos, y_pontos = construct_grids(-51.237390, -51.22057, -30.036518, -30.022946, 0.001)

In [37]:
x_pontos, y_pontos

(array([-51.236, -51.235, -51.234, -51.233, -51.232, -51.231, -51.23 ,
        -51.229, -51.228, -51.227, -51.226, -51.225, -51.224, -51.223,
        -51.222, -51.221, -51.22 , -51.219]),
 array([-30.0355, -30.0345, -30.0335, -30.0325, -30.0315, -30.0305,
        -30.0295, -30.0285, -30.0275, -30.0265, -30.0255, -30.0245,
        -30.0235, -30.0225, -30.0215, -30.0205, -30.0195]))

In [38]:
x_grid, y_grid = np.meshgrid(x_pontos,y_pontos)

In [39]:
array_sum = np.sum(x_grid)
array_has_nan = np.isnan(array_sum)

print(array_has_nan)

False


In [40]:
array_sum = np.sum(y_grid)
array_has_nan = np.isnan(array_sum)

print(array_has_nan)

False


In [41]:
zip (x_grid, y_grid)

In [42]:
Coord_test = pd.read_excel('Coord.xlsx')
Coord_test

,lat,lng
0,-30.0355,-51.236
1,-30.0345,-51.236
2,-30.0335,-51.236
3,-30.0325,-51.236
4,-30.0315,-51.236
...,...,...
284,-30.0235,-51.220
285,-30.0225,-51.220
286,-30.0215,-51.220
287,-30.0205,-51.220


Above i created a array with the data to test and an dataframe to plot the graph points

In [43]:
poa_map = folium.Map(location=[latitude, longitude], zoom_start=15)
# generate map centered around Porto Alegre

for lat, lng in zip(Coord_test.lat, Coord_test.lng):
    folium.features.CircleMarker(
        [lat, lng],
        radius=1,
        color='Purple',
        fill = False,
            ).add_to(poa_map)
    
poa_map
    

### 4. Modelling

#### Now i'll train and test the data using kernel density estimate of the distribution

In [44]:
from sklearn.neighbors import KernelDensity

In [45]:
array_teste = Coord_test.to_numpy()
array_teste

array([[-30.0355, -51.236 ],
       [-30.0345, -51.236 ],
       [-30.0335, -51.236 ],
       [-30.0325, -51.236 ],
       [-30.0315, -51.236 ],
       [-30.0305, -51.236 ],
       [-30.0295, -51.236 ],
       [-30.0285, -51.236 ],
       [-30.0275, -51.236 ],
       [-30.0265, -51.236 ],
       [-30.0255, -51.236 ],
       [-30.0245, -51.236 ],
       [-30.0235, -51.236 ],
       [-30.0225, -51.236 ],
       [-30.0215, -51.236 ],
       [-30.0205, -51.236 ],
       [-30.0195, -51.236 ],
       [-30.0355, -51.235 ],
       [-30.0345, -51.235 ],
       [-30.0335, -51.235 ],
       [-30.0325, -51.235 ],
       [-30.0315, -51.235 ],
       [-30.0305, -51.235 ],
       [-30.0295, -51.235 ],
       [-30.0285, -51.235 ],
       [-30.0275, -51.235 ],
       [-30.0265, -51.235 ],
       [-30.0255, -51.235 ],
       [-30.0245, -51.235 ],
       [-30.0235, -51.235 ],
       [-30.0225, -51.235 ],
       [-30.0215, -51.235 ],
       [-30.0205, -51.235 ],
       [-30.0195, -51.235 ],
       [-30.03

In [46]:
t = array_teste

In [47]:
# construct a spherical kernel density estimate of the distribution
kde = KernelDensity(bandwidth=0.6, metric='haversine')
kde.fit(np.radians(c))
tc = kde.score_samples(np.radians(t))


In [48]:
# construct a spherical kernel density estimate of the distribution
kde = KernelDensity(bandwidth=0.6, metric='haversine')
kde.fit(np.radians(b))
tb = kde.score_samples(np.radians(t))


#### This data is used to test the kde

Lets transform the Coord_test into array called array(t)

In [63]:
Kde_result = tc - tb
valores_corrigidos = Kde_result * 100000000
valores_corrigidos 

array([-2.09877107, -1.99447432, -1.89017739, -1.78588055, -1.68158363,
       -1.5772867 , -1.47298995, -1.36869307, -1.26439619, -1.16009917,
       -1.05580238, -0.9515055 , -0.84720861, -0.74291178, -0.63861489,
       -0.53431797, -0.43002109, -1.89100318, -1.78670412, -1.68240519,
       -1.57810631, -1.4738073 , -1.36950833, -1.26520936, -1.16091039,
       -1.05661142, -0.95231236, -0.84801335, -0.74371456, -0.63941554,
       -0.53511657, -0.43081756, -0.32651868, -0.22221971, -1.68323595,
       -1.57893494, -1.47463384, -1.37033274, -1.26603172, -1.16173067,
       -1.05742957, -0.95312855, -0.8488275 , -0.74452644, -0.64022534,
       -0.53592424, -0.43162327, -0.32732212, -0.22302116, -0.1187201 ,
       -0.01441882, -1.4754697 , -1.37116651, -1.26686333, -1.16256014,
       -1.05825704, -0.95395363, -0.84965066, -0.74534743, -0.64104428,
       -0.53674105, -0.432438  , -0.32813485, -0.22383166, -0.11952848,
       -0.01522533,  0.08907786,  0.193381  , -1.26770394, -1.16

In [65]:
score_corrigido = pd.DataFrame(valores_corrigidos)
score_corrigido.columns = ['score']
score_corrigido



,score
0,-2.098771
1,-1.994474
2,-1.890177
3,-1.785881
4,-1.681584
...,...
284,2.477387
285,2.581717
286,2.686047
287,2.790378


### Let's suppose if Kernel Density estimation is positive, there are more coffee shops in area, if it's negative the most often are bars

Lets create a new map from Porto Alegre using the lat and lng created to test data.

In [66]:
matriz_final = pd.concat([Coord_test, score_corrigido], axis=1)
matriz_final

,lat,lng,score
0,-30.0355,-51.236,-2.098771
1,-30.0345,-51.236,-1.994474
2,-30.0335,-51.236,-1.890177
3,-30.0325,-51.236,-1.785881
4,-30.0315,-51.236,-1.681584
...,...,...,...
284,-30.0235,-51.220,2.477387
285,-30.0225,-51.220,2.581717
286,-30.0215,-51.220,2.686047
287,-30.0205,-51.220,2.790378


In [69]:
array_sum = np.sum(matriz_final)
array_has_nan = np.isnan(array_sum)

print(array_has_nan)

lat      False
lng      False
score    False
dtype: bool


In [57]:
from folium.plugins import HeatMap

result_kde = folium.Map(location=[latitude, longitude], zoom_start=16)
# generate map centered around Porto Alegre
result_kde

In [77]:
json_corrigido = pd.Series(valores_corrigidos).to_json(orient='values')
json_corrigido 

'[-2.098771068,-1.994474319,-1.8901773924,-1.7858805546,-1.6815836279,-1.5772867012,-1.4729899522,-1.36869307,-1.2643961878,-1.1600991723,-1.0558023789,-0.9515054966,-0.8472086144,-0.7429117765,-0.6386148943,-0.5343179676,-0.4300210854,-1.8910031763,-1.786704118,-1.6824051929,-1.5781063123,-1.4738072984,-1.369508329,-1.2652093595,-1.16091039,-1.0566114206,-0.9523123623,-0.8480133484,-0.7437145566,-0.6394155427,-0.5351165733,-0.4308175594,-0.3265186788,-0.2222197093,-1.6832359506,-1.5789349384,-1.4746338373,-1.3703327362,-1.2660317239,-1.1617306672,-1.0574295661,-0.9531285539,-0.8488274972,-0.7445264405,-0.6402253394,-0.5359242383,-0.4316232705,-0.327322125,-0.2230211571,-0.1187201004,-0.0144188217,-1.475469702,-1.3711665137,-1.2668633254,-1.162560137,-1.0582570376,-0.9539536272,-0.8496506609,-0.7453474282,-0.6410442843,-0.5367410516,-0.4324379965,-0.3281348526,-0.2238316643,-0.119528476,-0.0152253321,0.0890778562,0.1933810001,-1.2677039418,-1.1633987107,-1.0590934352,-0.9547881152,-0.8

In [129]:

teste_map = folium.Map(location=[latitude, longitude], zoom_start=15)


for lat, lng, label in zip(matriz_final.lat, matriz_final.lng, abs(matriz_final.score)):
    
    if label > 0: 
        color = 'green'
    else:
        color = 'blue'
    label = abs(label)
    
    
    folium.features.CircleMarker(
        [lat, lng],
        radius= label,
        popup="matriz_final.score" ,
        fill = True,
        fill_color = '#3186cc',
        fill_opacity=0.6,

    ).add_to(teste_map)
    
teste_map
    